# IBM Cloud Pak for Data - DataStax Demonstration

## Introduction
Welcome to the IBM Cloud Pak for Data Multi-Cloud Virtualization Hands on Lab. 

In this lab you analyze data from multiple data sources, from across multiple Clouds, without copying data into a warehouse.

This hands-on lab uses live databases, were data is “virtually” available through the IBM Cloud Pak for Data Virtualization Service. This makes it easy to analyze data from across your multi-cloud enterprise using tools like, Jupyter Notebooks, Watson Studio or your favorite reporting tool like Cognos.  

Take a minute to watch this introductory video to get an overview of what you will see in this live Hands on Lab system.
http://ibm.biz/DTEVirtualMultiCloud

### Where to find this sample online
You can find a copy of this notebook on GITHUB at https://github.com/Db2-DTE-POC/CPDDVLAB.

### The business problem and the landscape
The Acme Company needs timely analysis of stock trading data from multiple source systems. 

Their data science and development teams needs access to:
* Customer data
* Account data
* Trading data
* Stock history and Symbol data

The data sources are running on premises and on the cloud. In this example many of the databases are also running on OpenShift but they could be managed, virtual or bare-metal cloud installations. IBM Cloud Pak for Data doesn't care. Enterprise DB (Postgres) is also running in the Cloud. Mongo and Informix are running on premises. 

To simplify access for Data Scientists and Developers the Acme team wants to make all their data look like it is coming from a single database. They also want to combine data to create simple to use tables.

In the past, Acme built a dedicated data warehouse, and then created ETL (Export, Transform and Load) job to move data from each data source into the warehouse were it could be combined. Now they can just virtualize your data without moving it.

### Loading the Python Libraries
To submit SQL statements to Data Virtualization relies we use a Jupyter notebook extension, commonly refer to as a "magic" command, to connect to a Db2 database. To use the commands you load load the extension by running another notebook call db2 that contains all the required code 
<pre>
&#37;run db2.ipynb
</pre>
The cell below loads the Db2 extension directly from GITHUB. Note that it will take a few seconds for the extension to load, so you should generally wait until the "Db2 Extensions Loaded" message is displayed in your notebook. 
1. Click the cell below
2. Click **Run**. When the cell is finished running, In[*] will change to In[2]

In [ ]:
# !wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb
!wget -O db2.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/db2.ipynb

%run db2.ipynb
print('db2.ipynb loaded')

#### Connecting to Data Virtualization

Before any SQL commands can be issued, a connection needs to be made to the Data Virtualization system you will be using. 

The Db2 magic command tracks whether or not a connection has occured in the past and saves this information between notebooks and sessions. When you start up a notebook and issue a command, the program will reconnect to the database using your credentials from the last session. In the event that you have not connected before, the system will prompt you for all the information it needs to connect. This information includes:

- Database name
- Hostname
- PORT 
- Userid
- Password

#### Connecting to Data Virtualization SQL Engine
Run the cell below. Your DATAENGINEER username is automatically used. 

In [ ]:
# Connect to the IBM Cloud Pak for Data Virtualization Database from inside CPD
database = 'bigsql'
user = 'admin'
password = 'password'
host = '10.1.1.1'
port = '32601'

%sql CONNECT TO {database} USER {user} USING {password} HOST {host} PORT {port}

Now that you are connected to the Data Virtualization engine you can query the virtualized tables using all the power in the Db2 SQL query engine. 

There are five tables that are available in each of the data sources. The examples below use a variety of data sources. For simplicity the schema of each data source represents where the data comes from. For example the NETEZZA.STOCK_SYMBOLS table is a virtual table that retrieves it data from a NETEZZA database. For your own projects you can choose any schema. There is no requirement to choose a schema that represents the source. 

The first table is a list of customer accounts with the current total number of stock trading transactions and the current account balance.

In [ ]:
%sql -a select * from DATASTAXSPARK.STOCK_SYMBOLS ORDER BY SYMBOL FETCH FIRST 5 ROWS ONLY

The same table is also available from a different source. In this case the STOCK_SYMBOLS table from a virtualized CSV file is available as CSV.STOCK_SYMBOLS.

In [ ]:
%sql -a select * from CSV.STOCK_SYMBOLS ORDER BY SYMBOL FETCH FIRST 5 ROWS ONLY

To keep things simple, the virtualized data sources used in these examples have been encapsulated in views each using the TRADING schema. For example, the SQL below was used to map each virtualized table to a view. So you can create SQL to query the tables using the TRADING schema for each table. Later you can drop and recreate a view using a different source and keep the SQL you will run using the TRADING schema. 

    CREATE VIEW TRADING.CUSTOMERS AS SELECT * FROM CSV.CUSTOMER;
    CREATE VIEW TRADING.STOCK_SYMBOLS AS SELECT * FROM DATASTAXSPARK.STOCK_SYMBOLS;
    CREATE VIEW TRADING.STOCK_HISTORY_2 AS SELECT * FROM DB2WAREHOUSE.STOCK_HISTORY;
    CREATE VIEW TRADING.STOCK_TRANSACTIONS AS SELECT * FROM EDBONCPD.STOCK_TRANSACTIONS;
    CREATE VIEW TRADING.ACCOUNTS AS SELECT * FROM INFORMIX.ACCOUNTS;


Now, let's explore some of the queries that we can run to gain insight from these tables. Have a look at the structure of each of the five tables. The represent trading information for a set of customers for a stock brokerage firm. 

In [ ]:
%sql -a select * from TRADING.STOCK_SYMBOLS ORDER BY SYMBOL FETCH FIRST 5 ROWS ONLY

In [ ]:
%sql -a select * from TRADING.CUSTOMER FETCH FIRST 5 ROWS ONLY

In [ ]:
%sql -a select * from TRADING.STOCK_SYMBOLS FETCH FIRST 5 ROWS ONLY

In [ ]:
%sql -a select * from TRADING.STOCK_HISTORY FETCH FIRST 5 ROWS ONLY

In [ ]:
%sql -a select * from TRADING.STOCK_TRANSACTIONS FETCH FIRST 5 ROWS ONLY

In [ ]:
%sql -a select * from TRADING.STOCK_SYMBOLS FETCH FIRST 10 ROWS ONLY

### Building Insight from the Virtualized Tables
Once you have tables virtualized you can use all the power of SQL and the Db2 SQL Engine to build complex and rich queries of your data no mater where it was originally located.
#### Get Price of a Stock over the Year
Set the Stock Symbol in the line below and run the cell. This information is folded together with data coming from two identical tables, one on Db2 database and on on and Informix database. Run the next two cells. Then pick a new stock symbol from the list above, enter it into the cell below and run both cells again.

**Source Cloud Pak for Data - Db2 Warehouse**

In [ ]:
stock = 'AXP'
print('variable stock set to = ' + str(stock))

In [ ]:
%%sql -pl
SELECT WEEK(TX_DATE) AS WEEK, OPEN FROM DATASTAXSPARK.STOCK_HISTORY
WHERE SYMBOL = :stock AND TX_DATE != '2017-12-01'
ORDER BY WEEK(TX_DATE) ASC

#### Trend of Three Stocks
This chart shows three stock prices over the course of a year. It uses the same folded stock history information.

In [ ]:
stocks = ['INTC','MSFT','AAPL']

In [ ]:
%%sql -pl
SELECT SYMBOL, WEEK(TX_DATE), OPEN FROM TRADING.STOCK_HISTORY
WHERE SYMBOL IN (:stocks) AND TX_DATE != '2017-12-01'
ORDER BY WEEK(TX_DATE) ASC

#### 30 Day Moving Average of a Stock
Enter the Stock Symbol below to see the 30 day moving average of a single stock.

In [ ]:
stock = 'AAPL'

In [ ]:
sqlin = \
"""
SELECT WEEK(TX_DATE) AS WEEK, OPEN, 
     AVG(OPEN) OVER (
       ORDER BY TX_DATE
     ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING) AS MOVING_AVG
  FROM TRADING.STOCK_HISTORY
     WHERE SYMBOL = :stock
  ORDER BY WEEK(TX_DATE)
"""
df = %sql {sqlin}
txdate= df['WEEK']
sales = df['OPEN']
avg = df['MOVING_AVG']

plt.xlabel("Day", fontsize=12);
plt.ylabel("Opening Price", fontsize=12);
plt.suptitle("Opening Price and Moving Average of " + stock, fontsize=20);
plt.plot(txdate, sales, 'r');
plt.plot(txdate, avg, 'b');
plt.show();

#### Trading volume of INTC versus MSFT and AAPL in first week of November
Enter three stock symbols below.

In [ ]:
stocks = ['INTC','MSFT','AAPL']

In [ ]:
%%sql -pb
SELECT SYMBOL, DAY(TX_DATE), VOLUME/1000000 FROM TRADING.STOCK_HISTORY
WHERE SYMBOL IN (:stocks) AND WEEK(TX_DATE) =  45
ORDER BY DAY(TX_DATE) ASC

#### Show Stocks that Represent at least 3% of the Total Purchases during Week 45

In [ ]:
%%sql -pie
WITH WEEK45(SYMBOL, PURCHASES) AS (
  SELECT SYMBOL, SUM(VOLUME * CLOSE) FROM TRADING.STOCK_HISTORY
    WHERE WEEK(TX_DATE) =  45 AND SYMBOL <> 'DJIA'
  GROUP BY SYMBOL
),
ALL45(TOTAL) AS (
  SELECT SUM(PURCHASES) * .03 FROM WEEK45
)
SELECT SYMBOL, PURCHASES FROM WEEK45, ALL45
WHERE PURCHASES > TOTAL
ORDER BY SYMBOL, PURCHASES

### Stock Transaction Table
#### Show Transactions by Customer
This next two examples uses data folded together from three different data sources representing three different trading organizations to create a combined of a single customer's stock trades. 

**Source EDB Postgres on Cloud Pak for Data**

In [ ]:
%%sql -a
SELECT * FROM TRADING.STOCK_TRANSACTIONS
 WHERE CUSTID = '107196'
 FETCH FIRST 10 ROWS ONLY

#### Bought/Sold Amounts of Top 5 stocks 

In [ ]:
%%sql -a
WITH BOUGHT(SYMBOL, AMOUNT) AS
  (
  SELECT SYMBOL, SUM(QUANTITY) FROM TRADING.STOCK_TRANSACTIONS
  WHERE QUANTITY > 0
  GROUP BY SYMBOL
  ),
SOLD(SYMBOL, AMOUNT) AS
  (
  SELECT SYMBOL, -SUM(QUANTITY) FROM TRADING.STOCK_TRANSACTIONS
  WHERE QUANTITY < 0
  GROUP BY SYMBOL
  )
SELECT B.SYMBOL, B.AMOUNT AS BOUGHT, S.AMOUNT AS SOLD
FROM BOUGHT B, SOLD S
WHERE B.SYMBOL = S.SYMBOL
ORDER BY B.AMOUNT DESC
FETCH FIRST 5 ROWS ONLY

### Customer Accounts
#### Show Top 5 Customer Balance
These next two examples use data from an Informix database

**Source - Informix on Premises**

In [ ]:
%%sql -a
SELECT CUSTID, BALANCE FROM TRADING.ACCOUNTS
ORDER BY BALANCE DESC
FETCH FIRST 5 ROWS ONLY

#### Show Bottom 5 Customer Balance
**AWS - Db2, Azure - Postgres, Azure - Db2**

In [ ]:
%%sql -a
SELECT CUSTID, BALANCE FROM TRADING.ACCOUNTS
ORDER BY BALANCE ASC
FETCH FIRST 5 ROWS ONLY

### Selecting Customer Information from MongoDB
The MongoDB database (running on premises) has customer information in a document format. In order to materialize the document data as relational tables, a total of four virtual tables are generated by the Data Virtualization engine. The following query shows the tables that are generated for the Customer document collection and how to join them into a single relational table.

In [ ]:
%sql -a select TABSCHEMA, TABNAME, COLCOUNT from syscat.tables where TABSCHEMA = 'MONGO' and TABNAME like 'CUSTOMER%'

The tables are all connected through the CUSTOMERID field, which is based on the generated _id of the main CUSTOMER colllection. In order to reassemble these tables into a document, we must join them using this unique identifier. An example of the contents of the CUSTOMER_CONTACT table is shown below.

In [ ]:
%sql -a SELECT * FROM MONGO.CUSTOMERS_CONTACT FETCH FIRST 5 ROWS ONLY

A full document record is shown in the following SQL statement which joins all of the tables together.

In [ ]:
%%sql -a
SELECT C.CUSTOMERID AS CUSTID, 
       CI.FIRSTNAME, CI.LASTNAME, CI.BIRTHDATE,
       CC.CITY, CC.ZIPCODE, CC.EMAIL, CC.PHONE, CC.STREET, CC.STATE,
       CP.CARD_TYPE, CP.CARD_NO
FROM MONGO.CUSTOMERS C, MONGO.CUSTOMERS_CONTACT CC, 
     MONGO.CUSTOMERS_IDENTITY CI, MONGO.CUSTOMERS_PAYMENT CP
WHERE  CC.CUSTOMERS_ID = C."_ID" AND
       CI.CUSTOMERS_ID = C."_ID" AND
       CP.CUSTOMERS_ID = C."_ID"
FETCH FIRST 3 ROWS ONLY

### Joining Virtualized Data
In this final example we use data from four different data sources to answer a complex business question. "What are the names of the customers in Ohio, who bought the most during the highest trading day of the year (based on the Dow Jones Industrial Index)?" 

**Data Sources: Db2 Warehouse, Infomrix, MongoDB, Enterprise Postgres**

In [ ]:
%%sql -a
WITH MAX_VOLUME(AMOUNT) AS (
  SELECT MAX(VOLUME) FROM TRADING.STOCK_HISTORY
    WHERE SYMBOL = 'DJIA'
),
HIGHDATE(TX_DATE) AS (
  SELECT TX_DATE FROM TRADING.STOCK_HISTORY, MAX_VOLUME M
    WHERE SYMBOL = 'DJIA' AND VOLUME = M.AMOUNT
),
CUSTOMERS_IN_OHIO(CUSTID, LASTNAME) AS (
  SELECT C.CUSTOMERID, CI.LASTNAME
    FROM  MONGO.CUSTOMERS C, 
          MONGO.CUSTOMERS_CONTACT CC,
          MONGO.CUSTOMERS_IDENTITY CI
    WHERE CC.CUSTOMERS_ID = C."_ID" AND
          CI.CUSTOMERS_ID = C."_ID" AND
          CC.STATE = 'OH'
),
TOTAL_BUY(CUSTID,TOTAL) AS (
  SELECT C.CUSTID, SUM(SH.QUANTITY * SH.PRICE) 
    FROM CUSTOMERS_IN_OHIO C, TRADING.STOCK_TRANSACTIONS SH, HIGHDATE HD
  WHERE SH.CUSTID = C.CUSTID AND
        SH.TX_DATE = HD.TX_DATE AND 
        SH.QUANTITY > 0 
  GROUP BY C.CUSTID
)
SELECT C.LASTNAME, T.TOTAL 
  FROM CUSTOMERS_IN_OHIO C, TOTAL_BUY T
WHERE C.CUSTID = T.CUSTID
ORDER BY TOTAL DESC
FETCH FIRST 5 ROWS ONLY

### Seeing where your Virtualized Data is coming from
You may eventually work with a complex Data Virtualization schema with dozens or hundres of data sources. As an administrator or a Data Scientist you may need to understand where data is coming from. 

Fortunately, the Data Virtualization engine is based on Db2. It includes the same catalog of information as does a Db2 database with some additional features. If you want to work backwards and understand where each of your virtualized tables comes from. The list of virtualized tables is included in the **SYSCAT.NICKNAMES** catalog table. 

Run the following SQL to see all the virtual tables in your Data Virtualization system. We exclude the **DVSYS** schema since it contains system created virtual tables (nicknames), not user created virtual tables. 

In [ ]:
%%sql -a
SELECT TABSCHEMA, TABNAME
  FROM SYSCAT.NICKNAMES
    WHERE TABSCHEMA != 'DVSYS'
    ORDER BY TABSCHEMA, TABNAME

If you want to see exactly where a virtualized table comes from you can substitute the the schema and virtual table name in the following SQL procedure. For example the next cell retrieves the location of the NETEZZA.STOCK_SYMBOLS table. You can see the source schema name the source table name as well as the connection information.

In [ ]:
%%sql -a 
select * from table(dvsys.GET_VT_SOURCES('DATASTAXSPARK', 'STOCK_SYMBOLS'))

You can also find the same information in the Cloud Pak for Data user interface. Look for the **Metadata** option in the interface.

To see the source of all your virtual tables we just need to join the query and the procedure call.

In [ ]:
%%sql -a
SELECT N.TABSCHEMA AS TABSCHEMA, N.TABNAME AS TABNAME, S.SRCTABNAME AS SRCTABNAME, S.SRCSCHEMA AS SRCSCHEMA, S.SRCTYPE AS TYPE, S.DRIVER AS DRIVER, S.URL AS URL, S.USER AS USER, S.HOSTNAME AS HOSTNAME, S.PORT AS PORT, S.DBNAME AS DBNAME
  FROM SYSCAT.NICKNAMES N, TABLE(
  DVSYS.GET_VT_SOURCES(N.TABSCHEMA, N.TABNAME)) S
  WHERE N.TABSCHEMA != 'DVSYS'

As part of this lab, a view had already been created in the DV engine that encapsulates the query above. It is the **ADMIN.REMOTETABLESOURCE** view. Run the following SQL to see how it works. Notice that you can use it just like any table and qualify the results with a where or order by clause. In this example we are listing all the virtual tables that come from a Netezza source server.

Access has been granted to all users to this view. But you can choose to limit who can use it by granting or revoking permissions to it.

In [ ]:
%%sql -a 
SELECT TABSCHEMA, TABNAME 
    FROM ADMIN.REMOTETABLESOURCE 
    WHERE DRIVER = 'com.simba.spark.jdbc41.Driver'
    ORDER BY 'TABSCHEMA', 'TABNAME'

You can also find the which tables a view is dependent on by querying the SYSCAT.TABDEP table. Run the example below. There is a line in the result set for each table that a view is dependent on. For example in the results below, the TRADING.ACCOUNTS view is dependent on on virtual table: INFORMIX.ACCOUNTS. The OHIO query is much more complex and is dependent on a number of views and tables. For example it uses the TRADING.STOCK_HISTORY view which in turn pulls data from the DB2WAREHOUSE.STOCK_HISTORY table. Because of this both dependencies are listed.

In [ ]:
%%sql -a
select tabschema,
       tabname,
       bschema as dependent_schema,
       bname as dependent_name
from syscat.tabdep
where dtype = 'V'
      and tabschema not like 'SYS%'
      and tabschema = 'TRADING'
order by tabschema, tabname, dependent_schema, dependent_name;

### Check and Classify your Virtualized Data
The STOCK data has already been added to the Watson Knowledge catalog.
1. Select **Catalogs** and **All catalogs** from the main Cloud Pak for Data menu
2. Click **Default Catalog**
3. Enter **CUSTOMER** into the search bar
4. Click the **DATASTAXSPARK.CUSTOMER** table
5. Click the **Profile** tab. Scroll to the right to check the CARD_NO column. If profiling is complete you should see details on all the columns except for CARD_NO.  This includes the distribution of values in each column. Notice that the Credit Card Number column is unavailable because the data in the column is anonymized.

### Review the existing Data Protection Rules
Now we can check the rules that are already in place that control how users can access our virtualized data.
1. Select **Governace** and **Rules** from the main Cloud Pak for Data menu
<img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/Rules.png">
2. Click **Mask Credit Card Numbers**
<img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/MaskCreditCardNumbers.png">
3. Click **Edit**. While, you are not going to nake any changes to the rule, you can review the options.
<img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/RuleEdit.png">
4. Click **Cancel**

You can find out more about Data Protection Rules in the Cloud Pak for Data Knowledge Center: https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/governance/dmg_rules.html

### Preview Data in the Cloud Pak for Data Console
Once you data has been classified and profiled protection is in place. Following the established protection rule, the credit card number will be redacted (replaced with Xs) everywhere it is accessible through Cloud Pak for Data. As you saw in the last step, it is reacted when you, and other users, interact with it through Watson Knowledge Catalog. 

It is also redacted if you access the data through the Data Virtualization console. Complete the following steps to see an example:

First, switch to the LABUSER id to check that you can see the data you have just granted access for:
9. Click the user icon at the very top right of the console
10. Click **Log out**
11. Sign in using the LABUSER id 
12. Click the three bar menu at the top left of the IBM Cloud Pak for Data console
13. Select **Data Virtualization**

Next, preview the data your just protected:
1. Select **Data** and **Data virtualization** from the main Cloud Pak for Data menu
2. Select **My virtualized data** from the Data virtualization menu
3. Enter **CUSTOMER** in the **Find virtual objects** search
4. Click the **DATASTAXSPARK_CUSTOMER** table
5. Click the elipsis menu to the right of that row
6. Select **Preview**. The CARD_NO column should be redacted with Xs.
<img src="https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/media/MaskedVirtualTable.png">

### Test Data Protection
Importantly, the Credit Card information is protected when the data is access from an outside application. Let's reconnect to the Data Virtualization Engine as your DATAENGINEER user.

In [ ]:
# Connect to the IBM Cloud Pak for Data Virtualization Database from inside CPD
database = 'bigsql'
user = 'dataengineer1'
password = 'tsdvlab'
host = '10.1.1.1'
port = '32601'

%sql CONNECT TO {database} USER {user} USING {password} HOST {host} PORT {port}

Now we can run a test query

In [ ]:
%sql SELECT * FROM DATASTAXSPARK.CUSTOMER FETCH FIRST 10 ROWS ONLY;

You should see the CARD_NO column results only return Xs. Any application that accesses this data through the Data Virtualization engine will be redacted in the same way. Remember you are still connected through Python as the USER id.

## Connect to Data Virtualization Using RESTful Endpoint Services
The following is a brief example of how to use the Db2 11.5.4 RESTful Endpoint service to extend the capabilies of Cloud Pak for Data Virtualization. 

You can extend your Cloud Pak for Data system so that application programmers can create Representational State Transfer (REST) endpoints that can be used to interact with the Data Virtualization Service. 

Each endpoint is associated with a single SQL statement. Authenticated users of web, mobile, or cloud applications can use these REST endpoints from any REST HTTP client without having to install any Db2 drivers.

The Db2 REST server accepts an HTTP request, processes the request body, and returns results in JavaScript Object Notation (JSON).

The Db2 REST server is pre-installed and running on Docker on server7 (10.1.1.12) in the Demonstration cluster. As a programmer you can communicate with the service on port 50050. Your welcome note includes the external port you can use to interact with the Db2 RESTful Endpoint service directly.

You can find more information about this service at: https://www.ibm.com/support/producthub/db2/docs/content/SSEPGG_11.5.0/com.ibm.db2.luw.admin.rest.doc/doc/c_rest.html.

### Connecting to the RESTful Service

In [ ]:
# Import Libraries
import requests
import pandas as pd

# Define the Connection Header
headers = {
  "content-type": "application/json"
}

# Define the location of the RESTful Endpoint Service
Db2RESTful = "http://10.1.1.12:50050"

# Define the API call
API_Auth = "/v1/auth"

# Define the Data Virtualization Host on Cloud Pak for Data
body = {
  "dbParms": {
    "dbHost": "10.1.1.1",
    "dbName": "bigsql",
    "dbPort": 32601,
    "isSSLConnection": False,
    "username": "ADMIN",
    "password": "password"
  },
  "expiryTime": "300m"
}

# Try Connecting 
try:
    response = requests.post("{}{}".format(Db2RESTful,API_Auth), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))
print(response)
if (response.status_code == 200):
  token = response.json()["token"]
  print("Token: {}".format(token))
else: 
  print(response.json()["errors"])

# Using the token define a reusable header for subsequent calls
headers = {
  "authorization": f"{token}",
  "content-type": "application/json"
}

## Executing an SQL Statement
Executing SQL requires a different service endpoint. In this case we will use "/services/execsql"

In [ ]:
API_execsql = "/v1/services/execsql"

body = {
  "isQuery": True,
  "sqlStatement": "SELECT * FROM DATASTAXSPARK.STOCK_TRANSACTIONS WHERE CUSTID = '107196'",
  "sync": True
}

try:
    response = requests.post("{}{}".format(Db2RESTful,API_execsql), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))
    
print(response)

display(pd.DataFrame(response.json()['resultSet']))

### Create a RESTful Service
The most common way of interacting with the service is to fully encapsulate an SQL statement, including any parameters, in a unique RESTful service. This creates a secure separation between the database service and the RESTful programming service. It also allows you to create versions of the same service to make maintenance and evolution of programming models simple and predictable.

In [ ]:
#Define service call
API_makerest = "/v1/services"

body = {"isQuery": True,
       "parameters": [
         {
         "datatype": "VARCHAR(6)",
         "name": "@CUSTID"
         }
       ],
       "schema": "STOCK",
       "serviceDescription": "Get transactions given customer id",
       "serviceName": "gettransactions",
       "sqlStatement": "SELECT * FROM DATASTAXSPARK.STOCK_TRANSACTIONS WHERE CUSTID = @CUSTID",
       "version": "1.0"
}

# Try creating service
try:
    response = requests.post("{}{}".format(Db2RESTful,API_makerest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))
    
if (response.status_code == 201):
  print("Service Created")
else:
  print(response.json()['errors'])

## Call the new RESTful Service
Now you can call the RESTful service. In this case we will pass the stock symbol CAT. But like in the previous example you can try rerunning the service call with different stock symbols.

In [ ]:
API_runrest = "/v1/services/gettransactions/1.0"

body = {
  "parameters": {
    "@CUSTID": "107196"
  },
  "sync": True
}

try:
    response = requests.post("{}{}".format(Db2RESTful,API_runrest), headers=headers, json=body)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))
    
print(response)

display(pd.DataFrame(response.json()['resultSet']))

## Delete a Service
A single call is also available to delete a service. A response of 204 indicates success.

In [ ]:
API_deleteService = "/v1/services"
Service = "/gettransactions"
Version = "/1.0"

try:
    response = requests.delete("{}{}{}{}".format(Db2RESTful,API_deleteService,Service,Version), headers=headers)
except Exception as e:
    print("Unable to call RESTful service. Error={}".format(repr(e)))
    
print (response)

#### Credits: IBM 2021, Peter Kohlmann [kohlmann@ca.ibm.com]